<a href="https://colab.research.google.com/github/MrunmayMuduli/Capstone-Projects/blob/main/Assignment_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import pg8000
import requests

In [4]:
# Replace this part
# connection = pg8000.connect(
#     user=os.getenv("DB_USER"),
#     password=os.getenv("DB_PASSWORD"),
#     host=os.getenv("DB_HOST"),
#     port=int(os.getenv("DB_PORT")),
#     database=os.getenv("DB_NAME")
# )

# With actual credentials
connection = pg8000.connect(
    user="postgres",
    password="Pinga13579#",
    host="database-weather.cxa2cw2csogq.us-east-1.rds.amazonaws.com",  # e.g., 'database-weather.cxa2cw2csogq.us-east-1.rds.amazonaws.com'
    port=5432,  # Default PostgreSQL port
    database="sample_db"
)


In [5]:
venue_id = "1"  # Example venue ID
start_date = "2024-11-01"  # Example start date in YYYY-MM-DD format
end_date = "2024-11-02"  # Example end date in YYYY-MM-DD format


In [7]:
def fetch_weather_data(latitude, longitude, start_date, end_date):
    api_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "dewpoint_2m", "apparent_temperature",
            "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth"
        ]
    }
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        data = response.json()
        print("API Response:", json.dumps(data, indent=2))  # Print the API response for debugging
        return data.get("hourly")
    else:
        print(f"Failed to fetch data from API. Status Code: {response.status_code}")
        return None


In [11]:
import json
import requests
import pg8000
import os
from datetime import datetime, timedelta

def fetch_weather_data(latitude, longitude, start_date, end_date):
    api_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "dewpoint_2m", "apparent_temperature",
            "precipitation_probability", "precipitation", "rain", "showers", "snowfall", "snow_depth"
        ]
    }
    response = requests.get(api_url, params=params)
    if response.status_code == 200:
        data = response.json()
        print("API Response:", json.dumps(data, indent=2))  # Print the API response for debugging
        return data.get("hourly")
    else:
        print(f"Failed to fetch data from API. Status Code: {response.status_code}")
        return None


In [14]:

def main():
    venue_id = "1"  # Replace with actual venue ID as needed
    start_date = "2024-11-01"  # Replace with actual start date in YYYY-MM-DD format
    end_date = "2024-11-02"  # Replace with actual end date in YYYY-MM-DD format

    if not venue_id or not start_date or not end_date:
        print("Invalid input. Please provide venue_id, start_date, and end_date.")
        return

    # Connect to the RDS database
    try:
        connection = pg8000.connect(
            user="postgres",
            password="Pinga13579#",
            host="database-weather.cxa2cw2csogq.us-east-1.rds.amazonaws.com",
            port=5432,
            database="sample_db"
        )
        cursor = connection.cursor()

        # Fetch venue data for latitude and longitude
        cursor.execute("SELECT latitude, longitude FROM Venue WHERE id = %s", (venue_id,))
        venue = cursor.fetchone()
        if not venue:
            print("Venue not found.")
            return

        latitude, longitude = venue

        # Fetch weather data from Open Meteo API
        weather_data = fetch_weather_data(latitude, longitude, start_date, end_date)

        if weather_data:
            timestamps = weather_data.get("time")  # Assuming "time" contains the list of timestamps
            if not timestamps:
                print("No timestamps found in weather data.")
                return

            for i in range(len(timestamps)):
                date = timestamps[i]

                temperature_2m = weather_data.get("temperature_2m", [None])[i]
                relative_humidity_2m = weather_data.get("relative_humidity_2m", [None])[i]
                dewpoint_2m = weather_data.get("dewpoint_2m", [None])[i]
                apparent_temperature = weather_data.get("apparent_temperature", [None])[i]
                precipitation_probability = weather_data.get("precipitation_probability", [None])[i]
                precipitation = weather_data.get("precipitation", [None])[i]
                rain = weather_data.get("rain", [None])[i]
                showers = weather_data.get("showers", [None])[i]
                snowfall = weather_data.get("snowfall", [None])[i]
                snow_depth = weather_data.get("snow_depth", [None])[i]

                # Insert the data into the Weather table
                cursor.execute(
                    "INSERT INTO Weather (venue_id, datetime, temperature, humidity, dewpoint, apparent_temperature, precipitation_probability, precipitation, rain, showers, snowfall, snow_depth) "
                    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
                    (venue_id, date, temperature_2m, relative_humidity_2m, dewpoint_2m, apparent_temperature,
                     precipitation_probability, precipitation, rain, showers, snowfall, snow_depth)
                )
            connection.commit()
            print("Weather data inserted successfully.")
        else:
            print("Failed to fetch weather data.")

    except Exception as e:
        print(f"Error: {str(e)}")

    finally:
        if connection:
            cursor.close()
            connection.close()

if __name__ == "__main__":
    main()



API Response: {
  "latitude": 52.52,
  "longitude": 13.419998,
  "generationtime_ms": 0.1310110092163086,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 38.0,
  "hourly_units": {
    "time": "iso8601",
    "temperature_2m": "\u00b0C",
    "relative_humidity_2m": "%",
    "dewpoint_2m": "\u00b0C",
    "apparent_temperature": "\u00b0C",
    "precipitation_probability": "%",
    "precipitation": "mm",
    "rain": "mm",
    "showers": "mm",
    "snowfall": "cm",
    "snow_depth": "m"
  },
  "hourly": {
    "time": [
      "2024-11-01T00:00",
      "2024-11-01T01:00",
      "2024-11-01T02:00",
      "2024-11-01T03:00",
      "2024-11-01T04:00",
      "2024-11-01T05:00",
      "2024-11-01T06:00",
      "2024-11-01T07:00",
      "2024-11-01T08:00",
      "2024-11-01T09:00",
      "2024-11-01T10:00",
      "2024-11-01T11:00",
      "2024-11-01T12:00",
      "2024-11-01T13:00",
      "2024-11-01T14:00",
      "2024-11-01T15:00",
      "2024-11-0